## Predictors of Domestic Violence in Washington, D.C.

Contributors: Seoho Hahm, Allison Lee

### Table of Contents
1. <a href='#prob'>Problem Statement
2. <a href='#sources'>Data Sources</a>
3. <a href='#approach'>Approach</a>
4. <a href='#eda'>Exploratory Data Analysis</a>
5. <a href='#feature'>Feature Selection</a>
6. <a href='#reg'>Regression</a>
7. <a href='#analysis'>Findings and Analysis</a>
8. <a href='#concl'>Conclusions and Recommendations</a>

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFE
import sklearn
import statsmodels.stats.outliers_influence as smd
%run ../pyfiles/data_cleaning
%run ../pyfiles/regression

<a id='prob'></a>
### Problem Statement

This analysis seeks to understand which factors are most efficient in predicting domestic violence in cities such as Washington, D.C. 

Specifically, we look at the strength and contribution of different variables to incidents of domestic violence in Washington, D.C., and the potential interactions between variables. 

<a id='sources'></a>
### Data Sources

We use a dataset of 101 features and 431 observations from a study conducted by Caterina Goman from the Urban Institute, "Alcohol Availability, Type of Alcohol Establishment, Distribution Policies, and Their Relationship to Crime and Disorder in the District of Columbia, 2000-2006". Each observation represents a block in Washington, D.C. 

The original dataset is available here: https://www.icpsr.umich.edu/icpsrweb/NACJD/studies/25763/summary

<a id='approach'></a>
### Approach

We take a more statistical approach to this analysis to understand which factors are most efficient at predicting incidents of domestic violence in Washington, D.C. All of our features are continuous. 

We used Cook's D to assess leverage and outliers in our variables. 

We used three different methods for feature selection--recursive feature elimination, domain knowledge / forward selection, and Lasso regression. We ran models on the resulting features and compared model metrics. We selected our final model based on adjusted R squared metric. 

We then checked the residual plots to ensure they met our assumptions of normal distribution and homoscedasticity. 

Finally, we present our main findings and recommendations. 

<a id='eda'></a>
### Exploratory Data Analysis

In [2]:
df_orig = pd.read_csv('../data/data.tsv', sep = '\t')
df = clean_orig_dataset(df_orig)
X = df.drop('AVGDV', axis = 1)
y = df['AVGDV']

There appears to be a high degree of interactions between our variables, which is something to keep in mind throughout our analysis. For example, these scatterplots illustrate how race interacts with poverty in D.C. 

<a id='feature'></a>
### Feature Selection

As a first step, we removed the features that are directly correlated with our dependent variable (for example, domestic violence incidents on the weekend). We tried three methods of feature selection to come up with a subset of features on which to run a model. 

We then used Cook's D to identify variables that have outliers with high influence over our regression results. We eliminated those outliers from our dataset (i.e. dropped the blocks with outlier data). 

In [3]:
cooksddf = cooksd(df, list(df.columns))
cleaned_df = dropping_outliers(list(df.columns), df, cooksddf)

In [4]:
# Scale features
scaled_df = scale_dataset(cleaned_df)

In [5]:
X = scaled_df.drop(['AVGDV'], axis = 1)
y = scaled_df['AVGDV']

**Recursive Feature Elimination**

First, we tried recursive feature elimination. In our case, recursive feature elimination works by recursively selecting smaller and smaller sets of features. The importance of each feature is obtained through the linear regression coefficient attribute (we scaled the features before inputting them in the rfe model). After initially training on the original set of features, the rfe "prunes" the least important features and repeats this process until the desired number of features is reached. We set our desired number of features as 7 for ease of interpretation. 

In [6]:
rfe_df = recursive_feature_elimination(X, y)

In [7]:
rfe_df.corr()> 0.7

,VACANT,SAMEHOUSE,YOUNGPOP,RESSTAB,PUBHOUSPT,PHYSDIS0203,ON_SQMI
VACANT,True,False,False,False,False,False,False
SAMEHOUSE,False,True,False,True,False,False,False
YOUNGPOP,False,False,True,False,False,False,False
RESSTAB,False,True,False,True,False,False,False
PUBHOUSPT,False,False,False,False,True,False,False
PHYSDIS0203,False,False,False,False,False,True,False
ON_SQMI,False,False,False,False,False,False,True


**Lasso Regression**

After standardizing our features, we select a subset of features using Lasso Regression (an embedded method). We tried this approach because we are interested in the importance of the features, and there is a high degree of multicollinearity within our features given the size of our dataset. 

In [8]:
X_lasso = cleaned_df.drop('AVGDV', axis = 1)
y_lasso = cleaned_df['AVGDV']
lasso_df = run_lasso(cleaned_df, X_lasso, y_lasso)

In [9]:
lasso_df.corr() > 0.7

,VACANT,YOUNGPOP,CONCDIS,METRO_BG,PUBHOUSPT,PHYSDIS0203
VACANT,True,False,False,False,False,False
YOUNGPOP,False,True,True,False,False,False
CONCDIS,False,True,True,False,False,False
METRO_BG,False,False,False,True,False,False
PUBHOUSPT,False,False,False,False,True,False
PHYSDIS0203,False,False,False,False,False,True


We therefore further removed variables due to multicollinearity. 

**Forward Selection**

In this method, we started with all variables in the model, and selected a subset of variables with the lowest p-values. We then added these variables to our model one by one, until we reached a limit where one became insignificant. 

In [10]:
forward_df = forward_selection(scaled_df)

The correlation matrix shows that there are no variables with a correlation coefficient above 0.7.

In [12]:
forward_df.corr() > 0.7

,POV,AA_POP,UNEMPL,FEMALE
POV,True,False,False,False
AA_POP,False,True,False,False
UNEMPL,False,False,True,False
FEMALE,False,False,False,True


<a id='reg'><a/>
### Multiple Linear Regression

We employ multiple linear regression with average incidents of domestic violence over 2005 to 2006 as our dependent variable. Multiple linear regression models make the following assumptions:
 - a linear relationship between the predictor variables and the dependent variable
 - the residuals are normally distributed
 - independent variables are not highly correlated with each other
 - homoscedasticity of the variance of error terms when plotted against independent variables

We run three different models using the three outputs of our feature selection methods. 

We compared the models using the Adjusted R squared metric, which is adjusted for the number of variables in the model. Our findings are summarized below:


In [13]:
run_model(lasso_df, np.array(y).reshape(-1, 1))

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.553
Model:                            OLS   Adj. R-squared (uncentered):              0.547
Method:                 Least Squares   F-statistic:                              86.27
Date:                Wed, 04 Dec 2019   Prob (F-statistic):                    5.03e-70
Time:                        16:31:22   Log-Likelihood:                         -430.82
No. Observations:                 424   AIC:                                      873.6
Df Residuals:                     418   BIC:                                      897.9
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [14]:
run_model(rfe_df, y)

                                 OLS Regression Results                                
Dep. Variable:                  AVGDV   R-squared (uncentered):                   0.647
Model:                            OLS   Adj. R-squared (uncentered):              0.641
Method:                 Least Squares   F-statistic:                              109.0
Date:                Wed, 04 Dec 2019   Prob (F-statistic):                    4.10e-90
Time:                        16:31:24   Log-Likelihood:                         -381.08
No. Observations:                 424   AIC:                                      776.2
Df Residuals:                     417   BIC:                                      804.5
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [15]:
run_model(forward_df, y)

                                 OLS Regression Results                                
Dep. Variable:                  AVGDV   R-squared (uncentered):                   0.318
Model:                            OLS   Adj. R-squared (uncentered):              0.312
Method:                 Least Squares   F-statistic:                              49.04
Date:                Wed, 04 Dec 2019   Prob (F-statistic):                    7.57e-34
Time:                        16:31:26   Log-Likelihood:                         -520.38
No. Observations:                 424   AIC:                                      1049.
Df Residuals:                     420   BIC:                                      1065.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Plotting Residuals

### Findings and Analysis